In [1]:
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import imutils
import time
import dlib
import cv2
import winsound

In [2]:
def eye_aspect_ratio(eye):
    # Calcula las distancias euclidianas entre los dos conjuntos de
    # Señales verticales del ojo (x, y) -coordenadas
    #A = dist.euclidean(eye[1], eye[5])
    #B = dist.euclidean(eye[2], eye[4])
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    # Calcular la distancia euclidiana entre la horizontal
    C = dist.euclidean(eye[0], eye[3])
    # calcula AR del ojo
    ear = (A + B) / (2.0 * C)
    # devuelve AR del ojo
    return ear

In [3]:
mStart=48
mEnd=68
jStart=0
jEnd= 17
rlStart=17
rlEnd= 22
leStart=22
leEnd= 27
nStart=27
nEnd= 36

In [15]:
# Se definen dos constantes, una para la relación de aspecto del ojo para indicar
# el parpadeo y luego una segunda constante para el número de
# Frames en que el ojo debe estar por debajo del umbral
EYE_AR_THRESH = 0.13#0.3 valor que se considera ojo cerrado
EYE_AR_CONSEC_FRAMES = 1#2 #tiempo en que el ojo pasa cerrado
sleep1_FRAMES = 3*3#3*5
sleep2_FRAMES = 6*5#7*5
sleep3_FRAMES = 8*5#8*5
sleep4_FRAMES = 10*5#10*5
sleep01_FRAMES = 6#6
sleep00_FRAMES = 2#2
freq = 2500 # Set frequency To 2500 Hertz
dur = 1000 # Set duration To 1000 ms == 1 second

In [20]:
# contadores para parpadeo
COUNTER = 0
TOTAL = 0
sleep=0;
gradsleep=0
# Inicializar la detección de cara con la libreria dlib (HOG-based) y luego
# usa el predictor del hito facial
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat" #modelo entrenado que identifica las caras
predictor = dlib.shape_predictor(PREDICTOR_PATH)
#se selecionan los indices del los ojos
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]
(jStart,jEnd) = face_utils.FACIAL_LANDMARKS_IDXS["jaw"]
(reStart,reEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eyebrow"]
(leStart,leEnd)= face_utils.FACIAL_LANDMARKS_IDXS["left_eyebrow"]
(nStart,nEnd)= face_utils.FACIAL_LANDMARKS_IDXS["nose"]
#cap = cv2.VideoCapture('andresprueba2.wmv')
cap = cv2.VideoCapture(0)#para usar la camara
#cap = VideoStream(src=0).start() #si quiero usar la camara de la laptop
fileStream = False
time.sleep(0.1)

[INFO] loading facial landmark predictor...


In [21]:
while True:
    ret, frame = cap.read()
    #frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # detecta caras en la imagen en escala de grises
    rects = detector(gray, 0)
    # ciclo sobre las detecciones de la cara
    for rect in rects:
        # Determina las marcas faciales para la región de la cara, luego
        # Convierte el punto de referencia facial (x, y) - a coordenada NumPy
        #Array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        # Extrae las coordenadas de los ojos izquierdo y derecho y calcula
        # la relación de aspecto (AR) del ojo para ambos ojos
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        # media de AR para los dos ojos
        ear = (leftEAR + rightEAR) / 2.0 #0.26 para lo achinados
        print(ear)
        # hace la convex hull para los dos ojos
        # se dibujan los dos ojos
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        # Compruebe si la relación de aspecto del ojo está por debajo del parpadeo
        # Y si es así se incrementa el contador del marco intermitente
        # counter debe llegar a 10 para poder deerminar somnolencia
        if ear < EYE_AR_THRESH: #si lo ojos estan entreabiertos o cerrados contar 
            COUNTER += 1
            if COUNTER >= sleep1_FRAMES:
                winsound.Beep(freq, dur)
                gradsleep = 50
            # De lo contrario, la relación de aspecto del ojo no está por debajo del parpadeo
            # límite
        else:
            # Si los ojos estaban cerrados por un número suficiente de
            # Luego incrementar el número total de parpadeos</pre>
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                TOTAL += 1
             #reseteo del contador            
            COUNTER = 0
        # Dibuja el número total de destellos en el marco junto con
        # la relación de aspecto calculada del ojo para el marco
        cv2.putText(frame, "Parpadeo: {}".format(TOTAL), (3, 30),
           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Grado de Somnolencia (0:10): {:.2f}".format(gradsleep), (3, 50),
           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Counter: {}".format(COUNTER), (3, 70),
           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    # mostramos el frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # si pulsa q se rompe el ciclo
    if key == ord("q"):
        break
    # limpiamos un poco
cap.release()
cv2.destroyAllWindows()

0.2448602198876527
0.24275053169013963
0.24682710144964692
0.25071977785192984
0.2291606480351591
0.25507561646881094
0.23561625874409928
0.23699664301470513
0.2473110903196602
0.24370451374728272
0.06580285752898725
0.063799254483
0.22182606101161473
0.24299049437246248
0.23619070626053837
0.24344380826014905
0.25114714274478556
0.2755491953957289
0.2679120339912721
0.11441161669512065
0.12724830354321226
0.12554141837587143
0.2292148699593258
0.2504471882228186
0.24561046892891475
0.2607836070797594
0.25091178636324574
0.17893349934847008
0.1247123480915528
0.1288296794008172
0.24682218663852742
0.26058357702816737
0.2537526022847123
0.27769279454206497
0.26059550733533127
0.29573746896833186
0.27103531779809953
0.25064937239680124
0.25685753066297823
0.26394312971060024
0.0875375122114207
0.09951922782757346
0.11075145492599996
0.11838924484040982
0.11054480661661424
0.10297701236330689
0.10220457936425434
0.09523108721705681
0.108922302046978
0.11118974791671085
0.2572495190312818
